In [1]:
import kayak
kayak.kayak_tickets("SEA","LAX","12","15","2021","business",1,0)

finished


In [2]:
departure_airport_code = "LAX"
arrival_airport_code = "SFO"
year = "2021"
month = "12"
day = "17"
class_type = "business"
carry_on_bag_number = 1
checked_bag_number = 2

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup

#import pandas as pd
#import numpy as np
#import requests as rq

import time, datetime#, json
import re

root_url = "https://www.kayak.com/flights/"
request_url = root_url + str(departure_airport_code) + "-" + str(arrival_airport_code) + "/" + str(year) + "-" + str(month) + "-" + str(day) + "/"+ str(class_type) + "?fs=cfc=" + str(carry_on_bag_number) + ";stops=~0;bfc=" + str(checked_bag_number) + "&sort=bestflight_a"
file_name = str(departure_airport_code) + str(arrival_airport_code) + str(year) + str(month) + str(day) + str(class_type) + str(carry_on_bag_number) + str(checked_bag_number) 

bot = webdriver.Chrome(executable_path='assets/chromedriver.exe')
bot.get(request_url)

f = open("assets/"+file_name+".csv", "a", encoding="utf-8") 
f.write("carrier,aircraft,depart_time,arrival_time,duration,price,time \n") 
start = datetime.datetime.now() 

try:
    for i in range(10):
        moreButton = bot.find_element_by_class_name('moreButton')
        moreButton.click()
        time.sleep(5)
        i = i+1
except:
    pass

soup = BeautifulSoup(bot.page_source, 'html5lib')
flights = soup.find_all("div",class_="resultWrapper") 
time.sleep(15)

for flight in flights:
    try:
        carrier = flight.find("div",class_=re.compile("carrier-text")).text.replace("\n","")
        aircraft = flight.find("div",class_=re.compile("-aircraft-name")).text.replace("\n","")
        depart_time = flight.find("div",class_=re.compile("-departure-row")).find("span",class_=re.compile("time")).text.replace("\n","")
        arrival_time = flight.find("div",class_=re.compile("-arrival-row")).find("span",class_=re.compile("time")).text .replace("\n","")
        duration = flight.find("div",class_=re.compile("duration-text")).text.replace("\n","")
        price = flight.find("span","price option-text").find("span", "price-text").text.replace("\n","")

        record = "%s,%s,%s,%s,%s,%s,%s \n" % (carrier,aircraft,depart_time,arrival_time,duration,price,start)
        f.write(record)
    except:
        pass

f.close()
#bot.close()
print("finished")

C:\Users\I\AppData\Local\Temp/ipykernel_11184/565752567.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  bot = webdriver.Chrome(executable_path='assets/chromedriver.exe')
C:\Users\I\AppData\Local\Temp/ipykernel_11184/565752567.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  moreButton = bot.find_element_by_class_name('moreButton')


finished
